In [1]:
!gcloud config set project  bwh-midrc-rapid-res-1655321320

Updated property [core/project].


In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
  project_name = "bwh-midrc-rapid-res-1655321320"
  bucket_name = "midrc-analysis-bwh"
  bucket_path = "bpr-results/"

In [4]:
import os
import shutil
import glob
import json
import matplotlib.pyplot as plt
from matplotlib import transforms
from matplotlib.colors import ListedColormap
import numpy as np

In [5]:
!pip install dicomweb-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 4.2 MB/s 
     |████████████████████████████████| 3.1 MB 18.4 MB/s 
     |████████████████████████████████| 2.0 MB 44.5 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=1b7994f8e83cbc471908c6e2e8b860730b840a97ffe63e5dedc5d4fb54d2a3a4
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


In [6]:
#install nrrd
!pip install pynrrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 369 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


In [11]:
from google.cloud import storage

storage_client = storage.Client(project = project_name)
#bucket = storage_client.get_bucket(bucket_name)

series_instance_uids = []

#blobs = storage_client.list_blobs(bucket)
blobs = storage_client.list_blobs(bucket_name, prefix=bucket_path, delimiter='/')
for blob in blobs:
    bn = blob.name
    bs = bn.split('.')
    if bs[-1] == 'json':
      bss = bn.split('/')[1]
      bs3 = bss.split('.')
      bs4 = '.'.join(bs3[:-1])
      series_instance_uids.append(bs4)

print(series_instance_uids[:10])

['1.2.826.0.1.3680043.10.474.419639.105799060738901793068313281334', '1.2.826.0.1.3680043.10.474.419639.106364025147079899289440200715', '1.2.826.0.1.3680043.10.474.419639.149051607502633615235577977952', '1.2.826.0.1.3680043.10.474.419639.189346812051260775638656981947', '1.2.826.0.1.3680043.10.474.419639.192916356998524553834723357563', '1.2.826.0.1.3680043.10.474.419639.198735019931123383691750806063', '1.2.826.0.1.3680043.10.474.419639.249044315484665760654506668895', '1.2.826.0.1.3680043.10.474.419639.259584978733948574762940092562', '1.2.826.0.1.3680043.10.474.419639.269534881585852761235289087419', '1.2.826.0.1.3680043.10.474.419639.272117657178318732150423166273']


In [12]:
print(len(series_instance_uids))

495


In [11]:
path_downloaded = '/content/downloaded_data'
path_nifti = '/content/nifti_data'
path_json =  '/content/json_data'
path_nrrd = '/content/nrrd_data'

for x in [path_downloaded, path_nifti, path_json, path_nrrd]:
  if os.path.isdir(x):
    try:
      shutil.rmtree(x)
    except OSError as err:
      print("Error: %s : %s" % (x, err.strerror))  
  os.mkdir(x)


In [12]:
import nrrd
import pandas
df_lungseg = pandas.DataFrame(columns = ['SeriesUID','lungslice'])
df_seriesuid_have_lungseg = pandas.DataFrame(columns=['SeriesUID','z_spacing'])

idx = 0

for seriesuid in series_instance_uids:
  
  for x in [path_nrrd]:
    if os.path.isdir(x):
      try:
        shutil.rmtree(x)
      except OSError as err:
        print("Error: %s : %s" % (x, err.strerror))  
    os.mkdir(x)
    
  # try copying the segmentation 
  fn_nrrd = 'seg_%s.nrrd' % seriesuid
  file_gs_uri = "gs://%s/%sseg_%s.nrrd" % (bucket_name, bucket_path, seriesuid)
  cmd1 = "gsutil cp %s %s " % (file_gs_uri, path_nrrd)
  print(idx, seriesuid) #cmd1)
  idx+=1
  retval = os.system(cmd1)
  if retval !=0:
    print("    skip missing segmentation")
    continue
  
  s, segheader = nrrd.read(os.path.join(path_nrrd, fn_nrrd))
  z_spacing = segheader['space directions'][2][2]
  print(z_spacing)

  df_seriesuid_have_lungseg = df_seriesuid_have_lungseg.append({'SeriesUID': seriesuid, 'z_spacing': z_spacing}, ignore_index = True)

  l1_list = list(np.array(np.where((np.sum((s==1),axis=(0,1))>0)==True)[0]).astype(int))
  l2_list = list(np.array(np.where((np.sum((s==2),axis=(0,1))>0)==True)[0]).astype(int))
  lall_list = l1_list + l2_list
  if len(lall_list)>0:
    lall_list = sorted(list(set(lall_list)))
    print('   ', min(lall_list),max(lall_list))
    for l in lall_list:
      df_lungseg = df_lungseg.append({'SeriesUID':seriesuid, 'lungslice':l}, ignore_index =True)

# done loop over seriesuid
df_seriesuid_have_lungseg.to_csv('/content/seriesuid_have_lungseg.csv', index=False)
df_lungseg.to_csv('/content/lungseg_by_series.csv', index=False)
                            

0 1.2.826.0.1.3680043.10.474.419639.105799060738901793068313281334
2.5
    155 177
1 1.2.826.0.1.3680043.10.474.419639.106364025147079899289440200715
    skip missing segmentation
2 1.2.826.0.1.3680043.10.474.419639.149051607502633615235577977952
2.5
    15 131
3 1.2.826.0.1.3680043.10.474.419639.189346812051260775638656981947
    skip missing segmentation
4 1.2.826.0.1.3680043.10.474.419639.192916356998524553834723357563
2.5
    110 110
5 1.2.826.0.1.3680043.10.474.419639.198735019931123383691750806063
2.5
    142 183
6 1.2.826.0.1.3680043.10.474.419639.249044315484665760654506668895
    skip missing segmentation
7 1.2.826.0.1.3680043.10.474.419639.259584978733948574762940092562
2.5
    2 98
8 1.2.826.0.1.3680043.10.474.419639.269534881585852761235289087419
2.5
    155 258
9 1.2.826.0.1.3680043.10.474.419639.272117657178318732150423166273
2.5
    143 193
10 1.2.826.0.1.3680043.10.474.419639.281464894123439644671058369302
2.5
    166 268
11 1.2.826.0.1.3680043.10.474.419639.28204131641

In [18]:
bucket_path = "results-summary"

cmd1 = "gsutil cp '/content/seriesuid_have_lungseg.csv' gs://%s/%s" % (bucket_name, bucket_path)
print(cmd1)
os.system(cmd1)

cmd2 = "gsutil cp '/content/lungseg_by_series.csv' gs://%s/%s" % (bucket_name, bucket_path)
print(cmd2)
os.system(cmd2)

gsutil cp '/content/seriesuid_have_lungseg.csv' gs://midrc-analysis-bwh/results-summary
gsutil cp '/content/lungseg_by_series.csv' gs://midrc-analysis-bwh/results-summary


0

In [18]:
import pandas
# save body part to a single .csv
fn_out = '/content/bpr_by_series_slices.csv'
print(bucket_name, bucket_path)

df = pandas.DataFrame(columns = ['SeriesUID','bodypart','slice'])
k=0
m=0
storage_client = storage.Client()
blobs = storage_client.list_blobs(bucket_name)
for b in blobs:
  #print('raw', b.name)
  p = os.path.split(b.name)
  #print(p)
  # skip non-axial and folders
  # temporarily autoremove any slashes from bucket_path
  if p[0]==bucket_path.replace('/','') and p[1].split('.')[-1]=='json':
    k +=1
    seriesuid = '.'.join(p[1].split('.')[:-1])
    #print(b.name)
    contents = storage_client.bucket(bucket_name).blob(b.name).download_as_string()
    jdata = json.loads(contents)
    for (bodypart, zlist) in jdata['body part examined'].items():
      if len(zlist) >0:
        for zz in zlist:
          df = df.append( {'SeriesUID': seriesuid, 'bodypart': bodypart, 'slice':zz }, ignore_index = True)
    if k%10==0:
      print(k,' ', end='')
    if k%120==0:
        print('')
print('\nfiles processed', k)
print(df)
df.to_csv(fn_out, index=False)
#cmd = 'gsutil cp %s gs://%s/%s/results-summary/' % (fn_out, bucket_name,bucket_path)
#print(cmd)
#os.system(cmd)

midrc-analysis-bwh bpr-results/
10  20  30  40  50  60  70  80  90  100  110  120  
130  140  150  160  170  180  190  200  210  220  230  240  
250  260  270  280  290  300  310  320  330  340  350  360  
370  380  390  400  410  420  430  440  450  460  470  480  
490  
files processed 495
                                                SeriesUID bodypart  slice
0       1.2.826.0.1.3680043.10.474.419639.105799060738...     legs    0.0
1       1.2.826.0.1.3680043.10.474.419639.105799060738...     legs    1.0
2       1.2.826.0.1.3680043.10.474.419639.105799060738...     legs    2.0
3       1.2.826.0.1.3680043.10.474.419639.105799060738...     legs    3.0
4       1.2.826.0.1.3680043.10.474.419639.105799060738...     legs    4.0
...                                                   ...      ...    ...
127151  2.16.840.1.114274.1818.57636332238089745215203...     head  102.0
127152  2.16.840.1.114274.1818.57636332238089745215203...     head  103.0
127153  2.16.840.1.114274.1818.5763633223

In [45]:
import nrrd
ct, ctheader = nrrd.read(os.path.join(path_nrrd, fn_ct_nrrd))
print(ctheader)

OrderedDict([('type', 'uint8'), ('dimension', 3), ('space', 'left-posterior-superior'), ('sizes', array([512, 512, 133])), ('space directions', array([[ 0.671875,  0.      ,  0.      ],
       [ 0.      , -0.671875,  0.      ],
       [ 0.      ,  0.      ,  2.      ]])), ('kinds', ['domain', 'domain', 'domain']), ('encoding', 'gzip'), ('space origin', array([-173.6640625 ,  -42.3359375 , -288.20001221])), ('ITK_FileNotes', 'TE=2.2e+02;Time=1029.840'), ('aux_file', ''), ('bitpix', '16'), ('cal_max', '0'), ('cal_min', '0'), ('datatype', '4'), ('descrip', 'TE=2.2e+02;Time=1029.840'), ('dim[0]', '3'), ('dim[1]', '512'), ('dim[2]', '512'), ('dim[3]', '133'), ('dim[4]', '1'), ('dim[5]', '0'), ('dim[6]', '0'), ('dim[7]', '0'), ('dim_info', ''), ('intent_code', '0'), ('intent_name', ''), ('intent_p1', '0'), ('intent_p2', '0'), ('intent_p3', '0'), ('pixdim[0]', '-1'), ('pixdim[1]', '0.671875'), ('pixdim[2]', '0.671875'), ('pixdim[3]', '2'), ('pixdim[4]', '0'), ('pixdim[5]', '0'), ('pixdim[6]',

In [ ]:
fn_seg_nrrd = os.path.join(path_nrrd, 'seg_%s.nrrd' % seriesuid)
nrrd.write( fn_seg_nrrd, s,  header = ctheader)

In [ ]:
# Define slices with segmented lungs

l1_list = list(np.array(np.where((np.sum((s==1),axis=(0,1))>0)==True)[0]).astype(int))
l1_min = min(l1_list)
l1_max = max(l1_list)
#print(l1_min, l1_max)

l2_list = list(np.array(np.where((np.sum((s==2),axis=(0,1))>0)==True)[0]).astype(int))
l2_min = min(l2_list)
l2_max = max(l2_list)
#print(l2_min, l2_max)


lmin_all = min([l1_min, l1_max,l2_min, l2_max])
lmax_all = max([l1_min, l1_max,l2_min, l2_max])
llist_out = [lmin_all, lmax_all]
lungs = list(range(lmin_all, lmax_all+1, 1))

print(llist_out)
print(lungs)

[155, 177]
[155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177]


In [ ]:
# Use BPR json for slice classification  

file_json = '%s.json' % seriesuid
cmd = "gsutil cp gs://%s/%s%s  %s" % (bucket_name, bucket_path, file_json, path_json)
print(cmd)
os.system(cmd)

fileObject = open("%s/%s" % (path_json, file_json), "r")
jsonContent = fileObject.read()
aList = json.loads(jsonContent)

for (bodypart, zlist) in aList['body part examined'].items():
    print(bodypart, zlist)

In [ ]:
for seriesuid in series_instance_uids:
  

  file_json = '%s.json' % seriesuid
  cmd = "gsutil cp gs://%s/%s%s  %s" % (bucket_name, bucket_path, file_json, path_json)
  print(cmd)
  os.system(cmd)

In [ ]:
k = 0

for seriesuid in series_instance_uids:

  file_json = seriesuid + '.json'

  fileObject = open("%s/%s" % (path_json, file_json), "r")
  jsonContent = fileObject.read()
  aList = json.loads(jsonContent)

  bp = aList['body part examined']
  ch = bp['chest']
  l = len(ch)
  if l > 0:
    k=k+1

print(k)

479


In [ ]:
file_gs_uri = "gs://%s/%s/%s.json" % (bucket_name, bucket_path, seriesuid)

json_list = glob(os.path.join( pat_dir_path_json,'*.json'))
  print(len(json_list), 'files for this patient')

  for jn in json_list:
    fileObject = open(jn, "r")
    jsonContent = fileObject.read()
    aList = json.loads(jsonContent)

    #print(aList['body part examined'])

    jj = os.path.split(jn)
    jjp = jj[0] # path
    jjn = jj[1] # json file name
    bpfn = os.path.join(jjp, jjn.split('.')[0]+'_bodypart_regr.txt')

    f = open(bpfn,'w')
    for (bodypart, zlist) in aList['body part examined'].items():
      #print(jn)
      print(bodypart, zlist)
      if (len(zlist)>0):
        f.write(bodypart+'\n')
    f.close()
